In [ ]:
import mlflow

import pandas as pd
import numpy as np

from scipy.stats import kurtosis
from scipy.fftpack import fft, ifft, fftshift
from scipy.signal import (
    spectrogram,
    welch,
    ShortTimeFFT,
    stft
)

from DataLoader import (
    loader,
    config
)

from Processer import preprocesser

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def spectral_kurtosis(signal, fs, window_size, overlap):
    f, t, Zxx = stft(signal, fs=fs, nperseg=window_size, noverlap=int(window_size * overlap))
    
    kurtosis_values = np.mean(np.power(np.abs(Zxx), 4), axis=0) / np.power(np.mean(np.power(np.abs(Zxx), 2), axis=0), 2) - 2
    
    return f, t, kurtosis_values


In [ ]:
df = pd.read_csv('../data/raw/spectr/5_2.csv', sep=';', skiprows=16)
# Проверяется на одной выборке
# df.drop('DateTime', axis=1, inplace=True)
# df = loader.fill_empty(loader.transform_header(df))
# Compress signal
# compressed = preprocesser.compress(df, floor='10min', method='mean')

In [ ]:
df

In [ ]:
df['Signals'] = df['Signals'].str.replace(',', '.').astype(float)

In [ ]:
kurtosis(df['Signals'])

In [ ]:
x = df['Signals'].to_numpy()

# Z-form:
# X(z) = 1 + 2z^(-1)+z^(-3), where z = exp(-jwT)

# Forward FFT
N = 30193

xFFT = fft(x, N)
# Magnitude spectrum
xA = np.abs(xFFT)
# Phase spectrum
xF = np.angle(xFFT)

# Complex
xReal = np.real(xFFT)
xImag = np.imag(xFFT)

# List of signals
xT = [x, xA, xF, np.array([xReal, xImag]).T]
lst_title = ['Signal', 'Spectrum', 'Phase', 'Real / Imag']

# Plot results
# fig = plt.figure(figsize=(12, 6), dpi=80)
# for i, sig in enumerate(xT):
#     plt.subplot(2, 2, i+1)
#     plt.ylabel('Level')
#     plt.title(lst_title[i])
#     if i == 0:
#         plt.stem(sig, basefmt='C0')
#         plt.xlabel('Time samples')
#         plt.xlim([-0.25, len(x)-0.75])
#     else:
#         plt.plot(sig)
#         plt.xlabel('Freq samples')
#         plt.xlim([0, N-1])
#     plt.grid()
# plt.tight_layout()
plt.plot(xA)

In [ ]:
# plt.plot(spectrogram(x, 30193, scaling='spectrum')[1])
f, t, Sxx = spectrogram(x, 30193.5)
plt.pcolormesh(t, f, Sxx, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

In [ ]:
spectral_kurtosis(x, 30193, 128), kurtosis(x)

In [ ]:
fs = 30193.5 # частота дискретизации (в Гц)
t = np.arange(0, 1, 1/fs)  # временная шкала
signal = x  # Пример сигнала
window_size = 100
overlap = .1
f, t, kurtosis_values = spectral_kurtosis(signal, fs, window_size, overlap)
sns.lineplot(x=t, y = kurtosis_values)